### Imports

In [77]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
from PIL import Image
import os

from img2vec_pytorch import Img2Vec

### Testing

In [111]:
# Initialize Img2Vec with GPU
img2vec_alexnet = Img2Vec(cuda=False, model="alexnet")  # default model resnet

# Read in an image (rgb format)
img = Image.open('test.png').convert('RGB')
# Get a vector from img2vec, returned as a torch FloatTensor
vec = img2vec_alexnet.get_vec(img, tensor=True)

print(vec.size())

# Initialize Img2Vec with GPU
img2vec_18 = Img2Vec(cuda=False, model="resnet-18")  # default model resnet

# Read in an image (rgb format)
img = Image.open('test.png').convert('RGB')
# Get a vector from img2vec, returned as a torch FloatTensor
vec = img2vec_18.get_vec(img, tensor=True)

print(vec.size())

torch.Size([1, 4096])
torch.Size([1, 512, 1, 1])


In [28]:
ref = Image.open('ref.png').convert('RGB')
ref_vec = img2vec.get_vec(ref, tensor=True)

p0 = Image.open('p0.png').convert('RGB')
p0_vec = img2vec.get_vec(p0, tensor=True)

p1 = Image.open('p1.png').convert('RGB')
p1_vec = img2vec.get_vec(p1, tensor=True)

In [29]:
# Using PyTorch Cosine Similarity
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(ref_vec,
              p0_vec)
print('\nCosine similarity: {0}\n'.format(cos_sim))


Cosine similarity: tensor([0.4113])



In [30]:
# Using PyTorch Cosine Similarity
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(ref_vec,
              p1_vec)
print('\nCosine similarity: {0}\n'.format(cos_sim))


Cosine similarity: tensor([0.3291])



### Common functions for loading dataset

In [41]:
def format_image_number(number):
    img_no_str = str(number)
    
    if len(img_no_str) == 1:
        im_no_name = "00000"
    elif len(img_no_str) == 2:
        im_no_name = "0000"
    elif len(img_no_str) == 3:
        im_no_name = "000"
    elif len(img_no_str) == 4:
        im_no_name = "00"
    elif len(img_no_str) == 5:
        im_no_name = "0"
    elif len(img_no_str) == 6:
        im_no_name = ""
        
    return im_no_name + img_no_str
    

def get_images_vector(path, number_of_images):  # returns a list of tensors
    # path of form: traditional/ref/
    feature_tensor_list = []
    
    for image_no in range(number_of_images):
        image_no_str = str(image_no)
        
        im_no_name = format_image_number(image_no_str) + ".png"
        
        path_to_image = path + im_no_name

        feature_vector_image = Image.open(path_to_image).convert('RGB')
        feature_vec = img2vec.get_vec(feature_vector_image, tensor=True)
        feature_tensor_list.append(feature_vec)
        
    return feature_tensor_list

In [71]:
def obtain_cosine_similarity_model_predictions(number_of_images, refs_vec_list, p0s_rec_list, p1s_rec_list):
    cosine_similarity_ref_and_p0 = []
    cosine_similarity_ref_and_p1 = []

    # list of predictions
    cosine_similarity_predictions = []
    
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    # cos_sim = cos(query_vector.unsqueeze(0), ref2_vector.unsqueeze(0))

    for image_no in range(number_of_images): 
        reshaped_ref = torch.reshape(refs_vec_list[image_no], (1,512))
        reshaped_p0 = torch.reshape(p0s_rec_list[image_no], (1,512))
        reshaped_p1 = torch.reshape(p1s_rec_list[image_no], (1,512))
        
        cosine_similarity_ref_and_p0.append(cos(reshaped_ref, reshaped_p0))
        cosine_similarity_ref_and_p1.append(cos(reshaped_ref, reshaped_p1))

        if cosine_similarity_ref_and_p0[image_no] >= cosine_similarity_ref_and_p1[image_no]:
            cosine_similarity_predictions.append(0)
        else:
            cosine_similarity_predictions.append(1)   
            
    return cosine_similarity_predictions

In [46]:
def obtain_prediction_accuracy(number_of_images, path_to_decision, actual_predictions):
    expected_decision_outputs = []

    for image_no in range(number_of_images):
            image_no_str = str(image_no)

            im_no_name = format_image_number(image_no_str) + ".npy"
            path_to_image = path_to_decision + im_no_name

            decision = np.load(path_to_image)

            if decision[0] <= 0.5: 
                expected_decision_outputs.append(0)
            else: 
                expected_decision_outputs.append(1)
                
    number_wrong_predictions = 0

    for image_no in range(number_of_images):
        if actual_predictions[image_no] != expected_decision_outputs[image_no]:
            number_wrong_predictions += 1

    accuracy = (number_of_images - number_wrong_predictions)/(number_of_images)
    
    return accuracy


In [61]:
def reshape_tensor_list(dim, tensor_list):
    reshaped_list = []
    for t in tensor_list:
        reshaped_t = torch.reshape(t, (1,dim))
        reshaped_list.append(t)
    return reshaped_list

### 1. Resnet

In [38]:
img2vec_res = Img2Vec(cuda=False, model="resnet-18")  # default model resnet

#### 1.1 Traditional

In [43]:
traditional_refs = get_images_vector("val/traditional/ref/", 4720)
traditional_p0s = get_images_vector("val/traditional/p0/", 4720)
traditional_p1s = get_images_vector("val/traditional/p1/", 4720)

In [44]:
print(len(traditional_refs))
print(len(traditional_p0s))
print(len(traditional_p1s))

4720
4720
4720


In [72]:
actual_model_predictions = obtain_cosine_similarity_model_predictions(4720, traditional_refs, traditional_p0s, traditional_p1s)

In [78]:
prediction_accuracy = obtain_prediction_accuracy(4720, "val/traditional/judge/", actual_model_predictions)

In [79]:
prediction_accuracy

0.8048728813559322

Prediction accuracy for Resnet-18 for traditional distortions: 0.8048728813559322

#### 1.2 CNN-based

In [81]:
cnn_refs = get_images_vector("val/cnn/ref/", 4720)
cnn_p0s = get_images_vector("val/cnn/p0/", 4720)
cnn_p1s = get_images_vector("val/cnn/p1/", 4720)

In [82]:
print(len(cnn_refs))
print(len(cnn_p0s))
print(len(cnn_p1s))

4720
4720
4720


In [86]:
actual_model_predictions_cnn_distortion = obtain_cosine_similarity_model_predictions(4720, cnn_refs, cnn_p0s, cnn_p1s)

In [87]:
prediction_accuracy_cnn_distortion = obtain_prediction_accuracy(4720, "val/cnn/judge/", actual_model_predictions_cnn_distortion)

In [88]:
prediction_accuracy_cnn_distortion

0.8442796610169492

Prediction accuracy for Resnet-18 for CNN-based distortions: 0.8442796610169492

#### 1.3 Colorization

In [90]:
color_refs = get_images_vector("val/color/ref/", 4720)
color_p0s = get_images_vector("val/color/p0/", 4720)
color_p1s = get_images_vector("val/color/p1/", 4720)

In [91]:
print(len(color_refs))
print(len(color_p0s))
print(len(color_p1s))

4720
4720
4720


In [92]:
actual_model_predictions_color_distortion = obtain_cosine_similarity_model_predictions(4720, color_refs, color_p0s, color_p1s)

In [93]:
prediction_accuracy_color_distortion = obtain_prediction_accuracy(4720, "val/color/judge/", actual_model_predictions_color_distortion)

In [94]:
prediction_accuracy_color_distortion

0.6533898305084745

Prediction accuracy for Resnet-18 for colorizaton distortions: 0.6533898305084745

#### 1.4 Deblurring --- IGNORE FOR NOW

In [95]:
deb_refs = get_images_vector("val/deblur/ref/", 9440)
deb_p0s = get_images_vector("val/deblur/p0/", 9440)
deb_p1s = get_images_vector("val/deblur/p1/", 9440)

fi_refs = get_images_vector("val/frameinterp/ref/", 1888)
fi_p0s = get_images_vector("val/frameinterp/p0/", 1888)
fi_p1s = get_images_vector("val/frameinterp/p1/", 1888)

In [96]:
print(len(deb_refs))
print(len(deb_p0s))
print(len(deb_p1s))

4720
4720
4720


In [97]:
actual_model_predictions_deb_distortion = obtain_cosine_similarity_model_predictions(4720, deb_refs, deb_p0s, deb_p1s)

In [98]:
prediction_accuracy_deb_distortion = obtain_prediction_accuracy(4720, "val/deblur/judge/", actual_model_predictions_deb_distortion)

In [99]:
prediction_accuracy_deb_distortion

0.6016949152542372

Prediction accuracy for Resnet-18 for deblurring distortions: 0.6016949152542372

#### 1.4, 1.5 and 1.6 Deblurring, frame interpolation and super resolution

In [100]:
deblur_refs = get_images_vector("val/deblur/ref/", 9440)
deblur_p0s = get_images_vector("val/deblur/p0/", 9440)
deblur_p1s = get_images_vector("val/deblur/p1/", 9440)
print("Deblurring is complete")

fi_refs = get_images_vector("val/frameinterp/ref/", 1888)
fi_p0s = get_images_vector("val/frameinterp/p0/", 1888)
fi_p1s = get_images_vector("val/frameinterp/p1/", 1888)
print("FI is complete")

spr_refs = get_images_vector("val/superres/ref/", 10856)
spr_p0s = get_images_vector("val/superres/p0/", 10856)
spr_p1s = get_images_vector("val/superres/p1/", 10856)
print("Spr is complete")

Deblurring is complete
FI is complete
Spr is complete


In [102]:
actual_model_predictions_blur_distortion = obtain_cosine_similarity_model_predictions(9440, deblur_refs, deblur_p0s, deblur_p1s)
prediction_accuracy_blur_distortion = obtain_prediction_accuracy(9440, "val/deblur/judge/", actual_model_predictions_blur_distortion)
print("Deblur: ")
print(prediction_accuracy_blur_distortion)
print("----------------")
print("")
print("")

Deblur: 
0.6113347457627119
----------------




In [103]:
actual_model_predictions_frame = obtain_cosine_similarity_model_predictions(1888, fi_refs, fi_p0s, fi_p1s)
prediction_accuracy_frame = obtain_prediction_accuracy(1888, "val/frameinterp/judge/", actual_model_predictions_frame)
print("Fi: ")
print(prediction_accuracy_frame)
print("----------------")
print("")
print("")

Fi: 
0.6604872881355932
----------------




In [108]:
actual_model_predictions_cnn_distortion_super = obtain_cosine_similarity_model_predictions(10856, spr_refs, spr_p0s, spr_p1s)
prediction_accuracy_cnn_distortion_super = obtain_prediction_accuracy(10856, "val/superres/judge/", actual_model_predictions_cnn_distortion_super)
print("Spr: ")
print(prediction_accuracy_cnn_distortion_super)
print("----------------")
print("")
print("")

Spr: 
0.7198784082535004
----------------




Prediction accuracy for Resnet-18 for deblurring distortions: 0.6113347457627119

Prediction accuracy for Resnet-18 for frame interpolation distortions: 0.6604872881355932

Prediction accuracy for Resnet-18 for super resolution distortions: 0.7198784082535004

### 2. AlexNet

In [112]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
from PIL import Image
import os

from img2vec_pytorch import Img2Vec

def format_image_number(number):
    img_no_str = str(number)
    
    if len(img_no_str) == 1:
        im_no_name = "00000"
    elif len(img_no_str) == 2:
        im_no_name = "0000"
    elif len(img_no_str) == 3:
        im_no_name = "000"
    elif len(img_no_str) == 4:
        im_no_name = "00"
    elif len(img_no_str) == 5:
        im_no_name = "0"
    elif len(img_no_str) == 6:
        im_no_name = ""
        
    return im_no_name + img_no_str
    
img2vec = Img2Vec(cuda=False, model="alexnet")  # default model resnet

def get_images_vector(path, number_of_images):  # returns a list of tensors
    # path of form: traditional/ref/
    feature_tensor_list = []
    
    for image_no in range(number_of_images):
        image_no_str = str(image_no)
        
        im_no_name = format_image_number(image_no_str) + ".png"
        
        path_to_image = path + im_no_name

        feature_vector_image = Image.open(path_to_image).convert('RGB')
        feature_vec = img2vec.get_vec(feature_vector_image, tensor=True)
        feature_tensor_list.append(feature_vec)
        
    return feature_tensor_list

def obtain_cosine_similarity_model_predictions(number_of_images, refs_vec_list, p0s_rec_list, p1s_rec_list):
    cosine_similarity_ref_and_p0 = []
    cosine_similarity_ref_and_p1 = []

    # list of predictions
    cosine_similarity_predictions = []
    
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    # cos_sim = cos(query_vector.unsqueeze(0), ref2_vector.unsqueeze(0))

    for image_no in range(number_of_images): 
        reshaped_ref = refs_vec_list[image_no]
        reshaped_p0 = p0s_rec_list[image_no]
        reshaped_p1 = p1s_rec_list[image_no]
        
        cosine_similarity_ref_and_p0.append(cos(reshaped_ref, reshaped_p0))
        cosine_similarity_ref_and_p1.append(cos(reshaped_ref, reshaped_p1))

        if cosine_similarity_ref_and_p0[image_no] >= cosine_similarity_ref_and_p1[image_no]:
            cosine_similarity_predictions.append(0)
        else:
            cosine_similarity_predictions.append(1)   
            
    return cosine_similarity_predictions

def obtain_prediction_accuracy(number_of_images, path_to_decision, actual_predictions):
    expected_decision_outputs = []

    for image_no in range(number_of_images):
            image_no_str = str(image_no)

            im_no_name = format_image_number(image_no_str) + ".npy"
            path_to_image = path_to_decision + im_no_name

            decision = np.load(path_to_image)

            if decision[0] <= 0.5: 
                expected_decision_outputs.append(0)
            else: 
                expected_decision_outputs.append(1)
                
    number_wrong_predictions = 0

    for image_no in range(number_of_images):
        if actual_predictions[image_no] != expected_decision_outputs[image_no]:
            number_wrong_predictions += 1

    accuracy = (number_of_images - number_wrong_predictions)/(number_of_images)
    
    return accuracy


def reshape_tensor_list(dim, tensor_list):
    reshaped_list = []
    for t in tensor_list:
        reshaped_t = torch.reshape(t, (1,dim))
        reshaped_list.append(t)
    return reshaped_list



print("Alexnet results:")

img2vec_res = Img2Vec(cuda=False, model="alexnet")  # default model resnet

traditional_refs = get_images_vector("val/traditional/ref/", 4720)
traditional_p0s = get_images_vector("val/traditional/p0/", 4720)
traditional_p1s = get_images_vector("val/traditional/p1/", 4720)

cnn_refs = get_images_vector("val/cnn/ref/", 4720)
cnn_p0s = get_images_vector("val/cnn/p0/", 4720)
cnn_p1s = get_images_vector("val/cnn/p1/", 4720)

color_refs = get_images_vector("val/color/ref/", 4720)
color_p0s = get_images_vector("val/color/p0/", 4720)
color_p1s = get_images_vector("val/color/p1/", 4720)

deblur_refs = get_images_vector("val/deblur/ref/", 9440)
deblur_p0s = get_images_vector("val/deblur/p0/", 9440)
deblur_p1s = get_images_vector("val/deblur/p1/", 9440)
print("Deblurring is complete")

fi_refs = get_images_vector("val/frameinterp/ref/", 1888)
fi_p0s = get_images_vector("val/frameinterp/p0/", 1888)
fi_p1s = get_images_vector("val/frameinterp/p1/", 1888)
print("FI is complete")

spr_refs = get_images_vector("val/superres/ref/", 10856)
spr_p0s = get_images_vector("val/superres/p0/", 10856)
spr_p1s = get_images_vector("val/superres/p1/", 10856)
print("Spr is complete")


actual_model_predictions = obtain_cosine_similarity_model_predictions(4720, traditional_refs, traditional_p0s, traditional_p1s)
prediction_accuracy = obtain_prediction_accuracy(4720, "val/traditional/judge/", actual_model_predictions)
print("Traditional: ")
print(prediction_accuracy)
print("----------------")
print("")
print("")

actual_model_predictions_cnn_distortion = obtain_cosine_similarity_model_predictions(4720, cnn_refs, cnn_p0s, cnn_p1s)
prediction_accuracy_cnn_distortion = obtain_prediction_accuracy(4720, "val/cnn/judge/", actual_model_predictions_cnn_distortion)
print("CNN: ")
print(prediction_accuracy_cnn_distortion)
print("----------------")
print("")
print("")


actual_model_predictions_color_distortion = obtain_cosine_similarity_model_predictions(4720, color_refs, color_p0s, color_p1s)
prediction_accuracy_color_distortion = obtain_prediction_accuracy(4720, "val/color/judge/", actual_model_predictions_color_distortion)
print("Colorization: ")
print(prediction_accuracy_color_distortion)
print("----------------")
print("")
print("")

actual_model_predictions_blur_distortion = obtain_cosine_similarity_model_predictions(9440, deblur_refs, deblur_p0s, deblur_p1s)
prediction_accuracy_blur_distortion = obtain_prediction_accuracy(9440, "val/deblur/judge/", actual_model_predictions_blur_distortion)
print("Deblur: ")
print(prediction_accuracy_blur_distortion)
print("----------------")
print("")
print("")


actual_model_predictions_frame = obtain_cosine_similarity_model_predictions(1888, fi_refs, fi_p0s, fi_p1s)
prediction_accuracy_frame = obtain_prediction_accuracy(1888, "val/frameinterp/judge/", actual_model_predictions_frame)
print("Fi: ")
print(prediction_accuracy_frame)
print("----------------")
print("")
print("")

actual_model_predictions_cnn_distortion_super = obtain_cosine_similarity_model_predictions(10856, spr_refs, spr_p0s, spr_p1s)
prediction_accuracy_cnn_distortion_super = obtain_prediction_accuracy(10856, "val/superres/judge/", actual_model_predictions_cnn_distortion_super)
print("Spr: ")
print(prediction_accuracy_cnn_distortion_super)
print("----------------")
print("")
print("")

Alexnet results:
Deblurring is complete
FI is complete
Spr is complete
Traditional: 
0.8108050847457627
----------------


CNN: 
0.8544491525423729
----------------


Colorization: 
0.6720338983050848
----------------


Deblur: 
0.639406779661017
----------------


Fi: 
0.666843220338983
----------------


Spr: 
0.7424465733235077
----------------




AlexNet: 
    
Traditional: 
0.8108050847457627


CNN: 
0.8544491525423729



Colorization: 
0.6720338983050848



Deblur: 
0.639406779661017



Fi: 
0.666843220338983



Spr: 
0.7424465733235077